In [1]:
!pip install pyscf openfermion

     |████████████████████████████████| 37.5 MB 1.2 MB/s 
     |████████████████████████████████| 1.0 MB 47.8 MB/s 
     |████████████████████████████████| 437 kB 38.2 MB/s 
     |████████████████████████████████| 1.6 MB 53.3 MB/s 
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13834 sha256=3d8c1a237ebf5281bcf15b584cb1d2288a0ba598e927c0e8793f734864082f40
  Stored in directory: /root/.cache/pip/wheels/7c/3d/8c/8192697412e9899dc55bbbb08bbc1197bef333caaa2a71c448
Successfully built pubchempy


https://github.com/quantumlib/OpenFermion-PySCF/blob/master/openfermionpyscf/_pyscf_molecular_data.py
https://github.com/quantumlib/OpenFermion/blob/v1.3.0/src/openfermion/chem/molecular_data.py#L222-L260


In [3]:
import numpy
from functools import reduce
from pyscf import ao2mo
from pyscf import scf
from pyscf.cc.addons import spatial2spin
from openfermion.chem import MolecularData


class PyscfMolecularData(MolecularData):

    """A derived class from openfermion.hamiltonians.MolecularData. This class
    is created to store the PySCF method objects as well as molecule data from
    a fixed basis set at a fixed geometry that is obtained from PySCF
    electronic structure packages. This class provides an interface to access
    the PySCF Hartree-Fock, MP, CI, Coupled-Cluster methods and their energies,
    density matrices and wavefunctions.
    Attributes:
        _pyscf_data(dict): To store PySCF method objects temporarily.
    """
    def __init__(self, geometry=None, basis=None, multiplicity=None,
                 charge=0, description="", filename="", data_directory=None):
        MolecularData.__init__(self, geometry, basis, multiplicity,
                               charge, description, filename, data_directory)
        self._pyscf_data = {}

    @property
    def canonical_orbitals(self):
        """Hartree-Fock canonical orbital coefficients (represented on AO
        basis).
        """
        if self._canonical_orbitals is None:
            scf = self._pyscf_data.get('scf', None)
            self._canonical_orbitals = scf.mo_coeff
        return self._canonical_orbitals

    @property
    def overlap_integrals(self):
        """Overlap integrals for AO basis."""
        if self._overlap_integrals is None:
            scf = self._pyscf_data.get('scf', None)
            self._overlap_integrals = scf.get_ovlp()
        return self._overlap_integrals

    @property
    def one_body_integrals(self):
        """A 2D array for one-body Hamiltonian (H_core) in the MO
        representation."""
        if self._one_body_integrals is None:
            scf = self._pyscf_data.get('scf', None)
            mo = self.canonical_orbitals
            h_core = scf.get_hcore()
            self._one_body_integrals = reduce(numpy.dot, (mo.T, h_core, mo))
        return self._one_body_integrals

    @property
    def two_body_integrals(self):
        """A 4-dimension array for electron repulsion integrals in the MO
        representation.  The integrals are computed as
        h[p,q,r,s]=\int \phi_p(x)* \phi_q(y)* V_{elec-elec} \phi_r(y) \phi_s(x) dxdy
        """
        if self._two_body_integrals is None:
            mol = self._pyscf_data.get('mol', None)
            mo = self.canonical_orbitals
            n_orbitals = mo.shape[1]

            eri = ao2mo.kernel(mol, mo)
            eri = ao2mo.restore(1, # no permutation symmetry
                                eri, n_orbitals)
            # See PQRS convention in OpenFermion.hamiltonians.molecular_data
            # h[p,q,r,s] = (ps|qr) = pyscf_eri[p,s,q,r]
            self._two_body_integrals = numpy.asarray(
                eri.transpose(0, 2, 3, 1), order='C')
        return self._two_body_integrals

    @property
    def cisd_one_rdm(self):
        r"""A 2-dimension array for CISD one-particle density matrix in the MO
        representation.  d[p,q] = < a^\dagger_p a_q >
        """
        if self._cisd_one_rdm is None:
            cisd = self._pyscf_data.get('cisd', None)
            if cisd is None:
                return None

            mf = self._pyscf_data.get('scf', None)
            if isinstance(mf, scf.uhf.UHF):
                raise ValueError('Spin trace for UCISD density matrix.')

            rdm1 = cisd.make_rdm1()
            if isinstance(mf, scf.rohf.ROHF):
                rdm1 = rdm1[0] + rdm1[1]

# pyscf one_rdm is computed as dm1[p,q] = <a^\dagger_q a_p>
            self._cisd_one_rdm = rdm1.T
        return self._cisd_one_rdm

    @property
    def cisd_two_rdm(self):
        r"""A 4-dimension array for CISD two-particle density matrix in the MO
        representation.  D[p,q,r,s] = < a^\dagger_p a^\dagger_q a_r a_s >
        """
        if self._cisd_two_rdm is None:
            cisd = self._pyscf_data.get('cisd', None)
            if cisd is None:
                return None

            mf = self._pyscf_data.get('scf', None)
            if isinstance(mf, scf.uhf.UHF):
                raise ValueError('Spin trace for UCISD density matrix.')

            rdm2 = cisd.make_rdm2()
            if isinstance(mf, scf.rohf.ROHF):
                aa, ab, bb = rdm2
                rdm2 = aa + bb + ab + ab.transpose(2, 3, 0, 1)

# pyscf.ci.cisd.make_rdm2 convention
#       dm2[p,s,q,r] = <a^\dagger_p a^\dagger_q a_r a_s>.
# the two_body_tensor in openfermion.ops._interaction_rdm.InteractionRDM
#       tbt[p,q,r,s] = <a^\dagger_p a^\dagger_q a_r a_s>.
            self._cisd_two_rdm = rdm2.transpose(0, 2, 3, 1)
        return self._cisd_two_rdm

    @property
    def ccsd_one_rdm(self):
        r"""A 2-dimension array for CCSD one-particle density matrix in the MO
        representation.  d[p,q] = < a^\dagger_p a_q >
        """
        ccsd = self._pyscf_data.get('ccsd', None)
        if ccsd is None:
            return None

        mf = self._pyscf_data.get('scf', None)
        if isinstance(mf, scf.uhf.UHF):
            raise ValueError('Spin trace for UCCSD density matrix.')

        rdm1 = ccsd.make_rdm1()
        if isinstance(mf, scf.rohf.ROHF):
            rdm1 = rdm1[0] + rdm1[1]
        return rdm1.T

    @property
    def ccsd_two_rdm(self):
        r"""A 4-dimension array for CCSD two-particle density matrix in the MO
        representation.  D[p,q,r,s] = < a^\dagger_p a^\dagger_q a_r a_s >
        """
        ccsd = self._pyscf_data.get('ccsd', None)
        if ccsd is None:
            return None

        mf = self._pyscf_data.get('scf', None)
        if isinstance(mf, scf.uhf.UHF):
            raise ValueError('Spin trace for UCCSD density matrix.')

        rdm2 = ccsd.make_rdm2()
        if isinstance(mf, scf.rohf.ROHF):
            aa, ab, bb = rdm2
            rdm2 = aa + bb + ab + ab.transpose(2, 3, 0, 1)
        return rdm2.transpose(0, 2, 3, 1)

    @property
    def mp2_one_rdm(self):
        r"""A 2-dimension array for MP2 one-particle density matrix in the MO
        representation.  d[p,q] = < a^\dagger_p a_q >
        """
        mp2 = self._pyscf_data.get('mp2', None)
        if mp2 is None:
            return None

        mf = self._pyscf_data.get('scf', None)
        if isinstance(mf, scf.uhf.UHF):
            raise ValueError('Spin trace for UMP2 density matrix.')

        rdm1 = mp2.make_rdm1()
        if isinstance(mf, scf.rohf.ROHF):
            rdm1 = rdm1[0] + rdm1[1]
        return rdm1.T

    @property
    def mp2_two_rdm(self):
        r"""A 4-dimension array for MP2 two-particle density matrix in the MO
        representation.  D[p,q,r,s] = < a^\dagger_p a^\dagger_q a_r a_s >
        """
        mp2 = self._pyscf_data.get('mp2', None)
        if mp2 is None:
            return None

        mf = self._pyscf_data.get('scf', None)
        if isinstance(mf, scf.uhf.UHF):
            raise ValueError('Spin trace for UMP2 density matrix.')

        rdm2 = mp2.make_rdm2()
        if isinstance(mf, scf.rohf.ROHF):
            aa, ab, bb = rdm2
            rdm2 = aa + bb + ab + ab.transpose(2, 3, 0, 1)
        return rdm2.transpose(0, 2, 3, 1)

    @property
    def fci_one_rdm(self):
        r"""A 2-dimension array for FCI one-particle density matrix in the MO
        representation.  d[p,q] = < a^\dagger_p a_q >
        """
        if self._fci_one_rdm is None:
            fci = self._pyscf_data.get('fci', None)
            if fci is None:
                return None

            mf = self._pyscf_data.get('scf', None)
            if isinstance(mf, scf.uhf.UHF):
                raise ValueError('Spin trace for UHF-FCI density matrices.')

            norb = self.canonical_orbitals.shape[1]
            nelec = self.n_electrons
            self._fci_one_rdm = fci.make_rdm1(fci.ci, norb, nelec).T
        return self._fci_one_rdm

    @property
    def fci_two_rdm(self):
        r"""A 4-dimension array for FCI two-particle density matrix in the MO
        representation.  D[p,q,r,s] = < a^\dagger_p a^\dagger_q a_r a_s >
        """
        if self._fci_two_rdm is None:
            fci = self._pyscf_data.get('fci', None)
            if fci is None:
                return None

            mf = self._pyscf_data.get('scf', None)
            if isinstance(mf, scf.uhf.UHF):
                raise ValueError('Spin trace for UHF-FCI density matrix.')

            norb = self.canonical_orbitals.shape[1]
            nelec = self.n_electrons
            fci_rdm2 = fci.make_rdm2(fci.ci, norb, nelec)
            self._fci_two_rdm = fci_rdm2.transpose(0, 2, 3, 1)
        return self._fci_two_rdm

    @property
    def ccsd_single_amps(self):
        r"""A 2-dimension array t[a,i] for CCSD single excitation amplitudes
        where a is virtual index and i is occupied index.
        """
        if self._ccsd_single_amps is None:
            ccsd = self._pyscf_data.get('ccsd', None)
            if ccsd is None:
                return None

            t1 = spatial2spin(ccsd.t1)
            no, nv = t1.shape
            nmo = no + nv
            self._ccsd_single_amps = numpy.zeros((nmo, nmo))
            self._ccsd_single_amps[no:,:no] = t1.T

        return self._ccsd_single_amps

    @property
    def ccsd_double_amps(self):
        r"""A 4-dimension array t[a,i,b,j] for CCSD double excitation amplitudes
        where a, b are virtual indices and i, j are occupied indices.
        """
        if self._ccsd_double_amps is None:
            ccsd = self._pyscf_data.get('ccsd', None)
            if ccsd is None:
                return None

            t2 = spatial2spin(ccsd.t2)
            no, nv = t2.shape[1:3]
            nmo = no + nv
            self._ccsd_double_amps = numpy.zeros((nmo, nmo, nmo, nmo))
            self._ccsd_double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

        return self._ccsd_double_amps

In [4]:
from functools import reduce

import numpy
from pyscf import gto, scf, ao2mo, ci, cc, fci, mp

from openfermion import MolecularData


def prepare_pyscf_molecule(molecule):
    """
    This function creates and saves a pyscf input file.
    Args:
        molecule: An instance of the MolecularData class.
    Returns:
        pyscf_molecule: A pyscf molecule instance.
    """
    pyscf_molecule = gto.Mole()
    pyscf_molecule.atom = molecule.geometry
    pyscf_molecule.basis = molecule.basis
    pyscf_molecule.spin = molecule.multiplicity - 1
    pyscf_molecule.charge = molecule.charge
    pyscf_molecule.symmetry = False
    pyscf_molecule.build()

    return pyscf_molecule


def compute_scf(pyscf_molecule):
    """
    Perform a Hartree-Fock calculation.
    Args:
        pyscf_molecule: A pyscf molecule instance.
    Returns:
        pyscf_scf: A PySCF "SCF" calculation object.
    """
    if pyscf_molecule.spin:
        pyscf_scf = scf.ROHF(pyscf_molecule)
    else:
        pyscf_scf = scf.RHF(pyscf_molecule)
    return pyscf_scf


def compute_integrals(pyscf_molecule, pyscf_scf):
    """
    Compute the 1-electron and 2-electron integrals.
    Args:
        pyscf_molecule: A pyscf molecule instance.
        pyscf_scf: A PySCF "SCF" calculation object.
    Returns:
        one_electron_integrals: An N by N array storing h_{pq}
        two_electron_integrals: An N by N by N by N array storing h_{pqrs}.
    """
    # Get one electrons integrals.
    n_orbitals = pyscf_scf.mo_coeff.shape[1]
    one_electron_compressed = reduce(numpy.dot, (pyscf_scf.mo_coeff.T,
                                                 pyscf_scf.get_hcore(),
                                                 pyscf_scf.mo_coeff))
    one_electron_integrals = one_electron_compressed.reshape(
        n_orbitals, n_orbitals).astype(float)

    # Get two electron integrals in compressed format.
    two_electron_compressed = ao2mo.kernel(pyscf_molecule,
                                           pyscf_scf.mo_coeff)

    two_electron_integrals = ao2mo.restore(
        1, # no permutation symmetry
        two_electron_compressed, n_orbitals)
    # See PQRS convention in OpenFermion.hamiltonians._molecular_data
    # h[p,q,r,s] = (ps|qr)
    two_electron_integrals = numpy.asarray(
        two_electron_integrals.transpose(0, 2, 3, 1), order='C')

    # Return.
    return one_electron_integrals, two_electron_integrals


def run_pyscf(molecule,
              run_scf=True,
              run_mp2=False,
              run_cisd=False,
              run_ccsd=False,
              run_fci=False,
              verbose=False):
    """
    This function runs a pyscf calculation.
    Args:
        molecule: An instance of the MolecularData or PyscfMolecularData class.
        run_scf: Optional boolean to run SCF calculation.
        run_mp2: Optional boolean to run MP2 calculation.
        run_cisd: Optional boolean to run CISD calculation.
        run_ccsd: Optional boolean to run CCSD calculation.
        run_fci: Optional boolean to FCI calculation.
        verbose: Boolean whether to print calculation results to screen.
    Returns:
        molecule: The updated PyscfMolecularData object. Note the attributes
        of the input molecule are also updated in this function.
    """
    # Prepare pyscf molecule.
    pyscf_molecule = prepare_pyscf_molecule(molecule)
    molecule.n_orbitals = int(pyscf_molecule.nao_nr())
    molecule.n_qubits = 2 * molecule.n_orbitals
    molecule.nuclear_repulsion = float(pyscf_molecule.energy_nuc())

    # Run SCF.
    pyscf_scf = compute_scf(pyscf_molecule)
    pyscf_scf.verbose = 0
    pyscf_scf.run()
    molecule.hf_energy = float(pyscf_scf.e_tot)
    if verbose:
        print('Hartree-Fock energy for {} ({} electrons) is {}.'.format(
            molecule.name, molecule.n_electrons, molecule.hf_energy))

    # Hold pyscf data in molecule. They are required to compute density
    # matrices and other quantities.
    molecule._pyscf_data = pyscf_data = {}
    pyscf_data['mol'] = pyscf_molecule
    pyscf_data['scf'] = pyscf_scf

    # Populate fields.
    molecule.canonical_orbitals = pyscf_scf.mo_coeff.astype(float)
    molecule.orbital_energies = pyscf_scf.mo_energy.astype(float)

    # Get integrals.
    one_body_integrals, two_body_integrals = compute_integrals(
        pyscf_molecule, pyscf_scf)
    molecule.one_body_integrals = one_body_integrals
    molecule.two_body_integrals = two_body_integrals
    molecule.overlap_integrals = pyscf_scf.get_ovlp()

    # Run MP2.
    if run_mp2:
        if molecule.multiplicity != 1:
            print("WARNING: RO-MP2 is not available in PySCF.")
        else:
            pyscf_mp2 = mp.MP2(pyscf_scf)
            pyscf_mp2.verbose = 0
            pyscf_mp2.run()
            # molecule.mp2_energy = pyscf_mp2.e_tot  # pyscf-1.4.4 or higher
            molecule.mp2_energy = pyscf_scf.e_tot + pyscf_mp2.e_corr
            pyscf_data['mp2'] = pyscf_mp2
            if verbose:
                print('MP2 energy for {} ({} electrons) is {}.'.format(
                    molecule.name, molecule.n_electrons, molecule.mp2_energy))

    # Run CISD.
    if run_cisd:
        pyscf_cisd = ci.CISD(pyscf_scf)
        pyscf_cisd.verbose = 0
        pyscf_cisd.run()
        molecule.cisd_energy = pyscf_cisd.e_tot
        pyscf_data['cisd'] = pyscf_cisd
        if verbose:
            print('CISD energy for {} ({} electrons) is {}.'.format(
                molecule.name, molecule.n_electrons, molecule.cisd_energy))

    # Run CCSD.
    if run_ccsd:
        pyscf_ccsd = cc.CCSD(pyscf_scf)
        pyscf_ccsd.verbose = 0
        pyscf_ccsd.run()
        molecule.ccsd_energy = pyscf_ccsd.e_tot
        pyscf_data['ccsd'] = pyscf_ccsd
        if verbose:
            print('CCSD energy for {} ({} electrons) is {}.'.format(
                molecule.name, molecule.n_electrons, molecule.ccsd_energy))

    # Run FCI.
    if run_fci:
        pyscf_fci = fci.FCI(pyscf_molecule, pyscf_scf.mo_coeff)
        pyscf_fci.verbose = 0
        molecule.fci_energy = pyscf_fci.kernel()[0]
        pyscf_data['fci'] = pyscf_fci
        if verbose:
            print('FCI energy for {} ({} electrons) is {}.'.format(
                molecule.name, molecule.n_electrons, molecule.fci_energy))

    # Return updated molecule instance.
    pyscf_molecular_data = PyscfMolecularData.__new__(PyscfMolecularData)
    pyscf_molecular_data.__dict__.update(molecule.__dict__)
    pyscf_molecular_data.save()
    return pyscf_molecular_data


def generate_molecular_hamiltonian(
        geometry,
        basis,
        multiplicity,
        charge=0,
        n_active_electrons=None,
        n_active_orbitals=None):
    """Generate a molecular Hamiltonian with the given properties.
    Args:
        geometry: A list of tuples giving the coordinates of each atom.
            An example is [('H', (0, 0, 0)), ('H', (0, 0, 0.7414))].
            Distances in angstrom. Use atomic symbols to
            specify atoms.
        basis: A string giving the basis set. An example is 'cc-pvtz'.
            Only optional if loading from file.
        multiplicity: An integer giving the spin multiplicity.
        charge: An integer giving the charge.
        n_active_electrons: An optional integer specifying the number of
            electrons desired in the active space.
        n_active_orbitals: An optional integer specifying the number of
            spatial orbitals desired in the active space.
    Returns:
        The Hamiltonian as an InteractionOperator.
    """

    # Run electronic structure calculations
    molecule = run_pyscf(
            MolecularData(geometry, basis, multiplicity, charge)
    )

    # Freeze core orbitals and truncate to active space
    if n_active_electrons is None:
        n_core_orbitals = 0
        occupied_indices = None
    else:
        n_core_orbitals = (molecule.n_electrons - n_active_electrons) // 2
        occupied_indices = list(range(n_core_orbitals))

    if n_active_orbitals is None:
        active_indices = None
    else:
        active_indices = list(range(n_core_orbitals,
                                    n_core_orbitals + n_active_orbitals))

    return molecule.get_molecular_hamiltonian(
            occupied_indices=occupied_indices,
            active_indices=active_indices)

In [6]:
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_sparse_operator #エラーが出る場合は openfermion を version 1.0.0 以上にしてみてください
from openfermion.chem import MolecularData

from scipy.optimize import minimize
from pyscf import fci
import numpy as np
import matplotlib.pyplot as plt

basis = "sto-3g"
multiplicity = 1
charge = 0
distance  = 0.977
geometry = [["H", [0,0,0]],["H", [0,0,distance]]]
description  = "tmp"
molecule = MolecularData(geometry, basis, multiplicity, charge, description)
molecule = run_pyscf(molecule,run_scf=1,run_fci=1)
n_qubit = molecule.n_qubits
n_electron = molecule.n_electrons
fermionic_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

In [7]:
jw_hamiltonian

(-0.3134960153409424+0j) [] +
(-0.04883472636540645+0j) [X0 X1 Y2 Y3] +
(0.04883472636540645+0j) [X0 Y1 Y2 X3] +
(0.04883472636540645+0j) [Y0 X1 X2 Y3] +
(-0.04883472636540645+0j) [Y0 Y1 X2 X3] +
(0.13978238294522732+0j) [Z0] +
(0.15762630551583418+0j) [Z0 Z1] +
(0.10745382591353002+0j) [Z0 Z2] +
(0.15628855227893645+0j) [Z0 Z3] +
(0.13978238294522727+0j) [Z1] +
(0.15628855227893645+0j) [Z1 Z2] +
(0.10745382591353002+0j) [Z1 Z3] +
(-0.13686895093682816+0j) [Z2] +
(0.16419290100986603+0j) [Z2 Z3] +
(-0.13686895093682816+0j) [Z3]

In [ ]:
mol = gto.M(atom='''
 Na 0. 0. 0.
 Na  0.  0.  1.''',
            basis={'Na':'lanl2dz'},
            ecp = {'Na':'lanl2dz'})
mf = scf.RHF(mol).run()
print(mf.mo_coeff)
print(mf.get_hcore())
print(ao2mo.kernel(mol,mf.mo_coeff))
# scf = self._pyscf_data.get('scf', None)
# mo = self.canonical_orbitals
# h_core = scf.get_hcore()
# molecule = mol


# pyscf_molecule = prepare_pyscf_molecule(molecule)
# molecule.n_orbitals = int(pyscf_molecule.nao_nr())
# molecule.n_qubits = 2 * molecule.n_orbitals
# molecule.nuclear_repulsion = float(pyscf_molecule.energy_nuc())

# # Run SCF.
# pyscf_scf = compute_scf(pyscf_molecule)
# pyscf_scf.verbose = 0
# pyscf_scf.run()
# molecule.hf_energy = float(pyscf_scf.e_tot)
# if verbose:
#     print('Hartree-Fock energy for {} ({} electrons) is {}.'.format(
#         molecule.name, molecule.n_electrons, molecule.hf_energy))

# # Hold pyscf data in molecule. They are required to compute density
# # matrices and other quantities.
# molecule._pyscf_data = pyscf_data = {}
# pyscf_data['mol'] = pyscf_molecule
# pyscf_data['scf'] = pyscf_scf

# # Populate fields.
# molecule.canonical_orbitals = pyscf_scf.mo_coeff.astype(float)
# molecule.orbital_energies = pyscf_scf.mo_energy.astype(float)

# # Get integrals.
# one_body_integrals, two_body_integrals = compute_integrals(
#     pyscf_molecule, pyscf_scf)
# molecule.one_body_integrals = one_body_integrals
# molecule.two_body_integrals = two_body_integrals
# molecule.overlap_integrals = pyscf_scf.get_ovlp()


https://github.com/quantumlib/OpenFermion/blob/v1.3.0/src/openfermion/chem/molecular_data.py#L222-L260

In [3]:

import openfermion.ops.representations as reps
from openfermion.config import EQ_TOLERANCE, DATA_DIRECTORY
import numpy 
from functools import reduce
from scipy.optimize import minimize
from pyscf import gto, scf, ao2mo, ci, cc, fci, mp
def spinorb_from_spatial(one_body_integrals, two_body_integrals):
    n_qubits = 2 * one_body_integrals.shape[0]

    # Initialize Hamiltonian coefficients.
    one_body_coefficients = numpy.zeros((n_qubits, n_qubits))
    two_body_coefficients = numpy.zeros(
        (n_qubits, n_qubits, n_qubits, n_qubits))
    # Loop through integrals.
    for p in range(n_qubits // 2):
        for q in range(n_qubits // 2):

            # Populate 1-body coefficients. Require p and q have same spin.
            one_body_coefficients[2 * p, 2 * q] = one_body_integrals[p, q]
            one_body_coefficients[2 * p + 1, 2 * q +
                                  1] = one_body_integrals[p, q]
            # Continue looping to prepare 2-body coefficients.
            for r in range(n_qubits // 2):
                for s in range(n_qubits // 2):

                    # Mixed spin
                    two_body_coefficients[2 * p, 2 * q + 1, 2 * r + 1, 2 *
                                          s] = (two_body_integrals[p, q, r, s])
                    two_body_coefficients[2 * p + 1, 2 * q, 2 * r, 2 * s +
                                          1] = (two_body_integrals[p, q, r, s])

                    # Same spin
                    two_body_coefficients[2 * p, 2 * q, 2 * r, 2 *
                                          s] = (two_body_integrals[p, q, r, s])
                    two_body_coefficients[2 * p + 1, 2 * q + 1, 2 * r +
                                          1, 2 * s +
                                          1] = (two_body_integrals[p, q, r, s])

    # Truncate.
    one_body_coefficients[
        numpy.absolute(one_body_coefficients) < EQ_TOLERANCE] = 0.
    two_body_coefficients[
        numpy.absolute(two_body_coefficients) < EQ_TOLERANCE] = 0.

    return one_body_coefficients, two_body_coefficients

def get_integrals(mf,mol):
    """Method to return 1-electron and 2-electron integrals in MO basis.
    Returns:
        one_body_integrals: An array of the one-electron integrals having
            shape of (n_orbitals, n_orbitals).
        two_body_integrals: An array of the two-electron integrals having
            shape of (n_orbitals, n_orbitals, n_orbitals, n_orbitals).
    Raises:
        MissingCalculationError: If integrals are not calculated.
    """
    # Make sure integrals have been computed.
    
    mo = mf.mo_coeff
    h_core = mf.get_hcore()
    one_body_integrals = reduce(numpy.dot, (mo.T, h_core, mo))
    n_orbitals = mo.shape[1]
    eri = ao2mo.kernel(mol, mo)
    eri = ao2mo.restore(1, # no permutation symmetry
                        eri, n_orbitals)
    # See PQRS convention in OpenFermion.hamiltonians.molecular_data
    # h[p,q,r,s] = (ps|qr) = pyscf_eri[p,s,q,r]
    two_body_integrals = numpy.asarray(
        eri.transpose(0, 2, 3, 1), order='C')
    return one_body_integrals, two_body_integrals

def get_active_space_integrals(mf,mol,
                                occupied_indices=None,
                                active_indices=None):
    """Restricts a molecule at a spatial orbital level to an active space
    This active space may be defined by a list of active indices and
        doubly occupied indices. Note that one_body_integrals and
        two_body_integrals must be defined
        n an orthonormal basis set.
    Args:
        occupied_indices: A list of spatial orbital indices
            indicating which orbitals should be considered doubly occupied.
        active_indices: A list of spatial orbital indices indicating
            which orbitals should be considered active.
    Returns:
        tuple: Tuple with the following entries:
        **core_constant**: Adjustment to constant shift in Hamiltonian
        from integrating out core orbitals
        **one_body_integrals_new**: one-electron integrals over active
        space.
        **two_body_integrals_new**: two-electron integrals over active
        space.
    """
    # Fix data type for a few edge cases
    occupied_indices = [] if occupied_indices is None else occupied_indices
    if (len(active_indices) < 1):
        raise ValueError('Some active indices required for reduction.')

    # Get integrals.
    one_body_integrals, two_body_integrals = get_integrals(mf,mol)
    return reps.get_active_space_integrals(one_body_integrals,
                                            two_body_integrals,
                                            occupied_indices, active_indices)

def get_molecular_hamiltonian(mf,mol,
                                occupied_indices=None,
                                active_indices=None):
    """Output arrays of the second quantized Hamiltonian coefficients.
    Args:
        occupied_indices(list): A list of spatial orbital indices
            indicating which orbitals should be considered doubly occupied.
        active_indices(list): A list of spatial orbital indices indicating
            which orbitals should be considered active.
    Returns:
        molecular_hamiltonian: An instance of the MolecularOperator class.
    Note:
        The indexing convention used is that even indices correspond to
        spin-up (alpha) modes and odd indices correspond to spin-down
        (beta) modes.
    """
    # Get active space integrals.
    if occupied_indices is None and active_indices is None:
        one_body_integrals, two_body_integrals = get_integrals(mf,mol)
        constant = mf.energy_nuc()
    else:
        core_adjustment, one_body_integrals, two_body_integrals = \
            get_active_space_integrals(mf,mol,occupied_indices, active_indices)
        constant = mf.energy_nuc() + core_adjustment

    one_body_coefficients, two_body_coefficients = spinorb_from_spatial(
        one_body_integrals, two_body_integrals)

    # Cast to InteractionOperator class and return.
    molecular_hamiltonian = reps.InteractionOperator(
        constant, one_body_coefficients, 1 / 2 * two_body_coefficients)

    return molecular_hamiltonian

In [12]:
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_sparse_operator #エラーが出る場合は openfermion を version 1.0.0 以上にしてみてください
from openfermion.chem import MolecularData
from functools import reduce
from scipy.optimize import minimize
from pyscf import gto, scf, ao2mo, ci, cc, fci, mp
import numpy as np
import matplotlib.pyplot as plt
mol = gto.M(atom='''
 Na 0. 0. 0.
 Na  0.  0.  1.''',
            basis={'Na':'lanl2dz'},
            ecp = {'Na':'lanl2dz'})
mf = scf.RHF(mol).run()
molecular_hamiltonian=get_molecular_hamiltonian(mf,mol,occupied_indices=None,active_indices=None)
fermionic_hamiltonian = get_fermion_operator(molecular_hamiltonian)
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

converged SCF energy = -0.0730796898485634


In [13]:
jw_hamiltonian

(20.554892876399347+0j) [] +
(-0.005550645745839964+0j) [X0 X1 Y2 Y3] +
(-0.005378603836969049+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Y15] +
(-0.004338885769824478+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Z17 Z18 Z19 Z20 Z21 Z22 Y23] +
(0.0003539977188736457+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Z17 Z18 Z19 Z20 Z21 Z22 Z23 Z24 Z25 Z26 Z27 Z28 Z29 Z30 Y31] +
(-0.005378603836969049+0j) [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 X14] +
(-0.004338885769824479+0j) [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Z17 Z18 Z19 Z20 Z21 X22] +
(0.0003539977188736457+0j) [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Z17 Z18 Z19 Z20 Z21 Z22 Z23 Z24 Z25 Z26 Z27 Z28 Z29 X30] +
(-0.0055181526251151785+0j) [X0 X1 Y4 Y5] +
(0.003293030640188194+0j) [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Y17] +
(-0.0048956570202927235+0j) [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Z17 Z18 Y19] +
(0.003293030640188194

In [4]:
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_sparse_operator #エラーが出る場合は openfermion を version 1.0.0 以上にしてみてください
from openfermion.chem import MolecularData
from functools import reduce
from scipy.optimize import minimize
from pyscf import gto, scf, ao2mo, ci, cc, fci, mp
import numpy as np
import matplotlib.pyplot as plt
mol = gto.M(atom='''
 Na 0. 0. 0.
 Na  0.  0.  1.''',
            basis={'Na':'lanl2dz'},
            # ecp = {'Na':'lanl2dz'}
            )
mf = scf.RHF(mol).run()
molecular_hamiltonian=get_molecular_hamiltonian(mf,mol,occupied_indices=None,active_indices=None)
fermionic_hamiltonian = get_fermion_operator(molecular_hamiltonian)
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)
print(jw_hamiltonian)

ストリーミング出力は最後の 5000 行に切り捨てられました。
(0.0003392566610253542+0j) [Y6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Y17 X18 Z19 Z20 Z21 Z22 Z23 Z24 Z25 Z26 Z27 Z28 X29] +
(-0.00017260306726205483+0j) [Y6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Y17 Y19 Z20 Z21 Z22 Z23 Z24 Z25 Z26 Z27 Y28] +
(-4.034395731287314e-05+0j) [Y6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Y17 X20 Z21 Z22 Z23 Z24 Z25 Z26 X27] +
(-0.0008304843657942521+0j) [Y6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Y17 Y21 Z22 Z23 Z24 Z25 Y26] +
(-1.020526125930749e-05+0j) [Y6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Y17 X28 Z29 Z30 X31] +
(-0.0002941509054638597+0j) [Y6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Y17 Y29 Y30] +
(0.0002876424153313881+0j) [Y6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Z17 X18 X20 Z21 Z22 Z23 Z24 Z25 Z26 Z27 Z28 Z29 Y30] +
(-0.0002834581411852789+0j) [Y6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Z17 X18 X26 Z27 Y28] +
(0.000456081279128022+0j) [Y6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Z15 Z16 Z17 Y18 X19 Z20 X21] +
(0.000456081279128022+0j) [Y6 Z7 Z8 Z9 

In [5]:
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_sparse_operator #エラーが出る場合は openfermion を version 1.0.0 以上にしてみてください
from openfermion.chem import MolecularData
from functools import reduce
from scipy.optimize import minimize
from pyscf import gto, scf, ao2mo, ci, cc, fci, mp
import numpy as np
import matplotlib.pyplot as plt
mol = gto.M(atom='''
 Na 0. 0. 0.
 Na  0.  0.  1.''',
            basis={'Na':'sto-3g'},
            # ecp = {'Na':'lanl2dz'}
            )
mf = scf.RHF(mol).run()
molecular_hamiltonian=get_molecular_hamiltonian(mf,mol,occupied_indices=None,active_indices=None)
fermionic_hamiltonian = get_fermion_operator(molecular_hamiltonian)
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)


converged SCF energy = -317.969824197392


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
print(jw_hamiltonian)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

